In [8]:
import pandas as pd
import numpy as np0

df = pd.read_csv("emissoes_carbono.csv")

def calcular_estatisticas_emissoes(filepath):
    try:
        df = pd.read_csv(filepath, encoding='utf-8')
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {filepath}")
        return None, 0
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return None, 0

    coluna_emissao = "emissão estimada de CO2"
    coluna_metrica = "métrica"

    if coluna_emissao not in df.columns:
        print(f"Erro: Coluna '{coluna_emissao}' não encontrada no CSV.")
        return None, 0
    if coluna_metrica not in df.columns:
        print(f"Erro: Coluna '{coluna_metrica}' não encontrada no CSV.")
        return None, 0

    # Tentar converter a coluna de emissão para numérico, tratando erros
    # Os valores já estão pré-processados, mas a verificação é útil
    
    # Assegurar que a coluna é tratada como string antes de substituir vírgulas por pontos
    df[coluna_emissao] = df[coluna_emissao].astype(str).str.replace(",", ".", regex=False)
    numeric_series = pd.to_numeric(df[coluna_emissao], errors='coerce')
    
    erros_conversao = numeric_series.isna().sum()
    df[coluna_emissao] = numeric_series.fillna(0)
    

    resultados_estatisticos = {}

    metricas_unicas = df[coluna_metrica].unique()
    if len(metricas_unicas) == 0:
        print("Nenhuma métrica encontrada para agrupar os dados.")
        return None, erros_conversao

    for metrica in metricas_unicas:
        df_filtrado = df[df[coluna_metrica] == metrica]
        if df_filtrado.empty:
            print(f"Nenhum dado encontrado para a métrica: {metrica}")
            continue

        dados_emissao_filtrados = df_filtrado[coluna_emissao]

        if dados_emissao_filtrados.empty:
            estatisticas = {
                "média": "N/A",
                "mediana": "N/A",
                "desvio_padrão": "N/A",
                "variância": "N/A",
                "amplitude": "N/A",
                "contagem_valores": 0
            }
        else:
            media = dados_emissao_filtrados.mean()
            mediana = dados_emissao_filtrados.median()
            desvio_padrao = dados_emissao_filtrados.std()
            variancia = dados_emissao_filtrados.var()
            # Calcular amplitude apenas se houver dados válidos para evitar erro com NaN
            amplitude = (dados_emissao_filtrados.max() - dados_emissao_filtrados.min()) if dados_emissao_filtrados.notna().any() and len(dados_emissao_filtrados) > 0 else 0
            
            estatisticas = {
                "média": f"{media:.2f}" if pd.notna(media) else "N/A",
                "mediana": f"{mediana:.2f}" if pd.notna(mediana) else "N/A",
                "desvio_padrão": f"{desvio_padrao:.2f}" if pd.notna(desvio_padrao) else "N/A",
                "variância": f"{variancia:.2f}" if pd.notna(variancia) else "N/A",
                "amplitude": f"{amplitude:.2f}" if pd.notna(amplitude) else "N/A",
                "contagem_valores": len(dados_emissao_filtrados)
            }
        
        resultados_estatisticos[metrica] = estatisticas

    return resultados_estatisticos, erros_conversao

if __name__ == "__main__":
    filepath = "emissoes_carbono.csv"
    resultados, erros = calcular_estatisticas_emissoes(filepath)

    if resultados:
        print("\n--- Resultados Estatísticos ---")
        for metrica, stats in resultados.items():
            print(f"\nEstatísticas para a métrica: '{metrica}'")
            print(f"  Número de observações: {stats['contagem_valores']}")
            print(f"  Média: {stats['média']}")
            print(f"  Mediana: {stats['mediana']}")
            print(f"  Desvio Padrão: {stats['desvio_padrão']}")
            print(f"  Variância: {stats['variância']}")
            print(f"  Amplitude (Máx - Mín): {stats['amplitude']}")




--- Resultados Estatísticos ---

Estatísticas para a métrica: 'Emissions (thousand metric tons of carbon dioxide)'
  Número de observações: 1722
  Média: 376398.09
  Mediana: 6559.00
  Desvio Padrão: 2316519.30
  Variância: 5366261658698.72
  Amplitude (Máx - Mín): 33147028.00

Estatísticas para a métrica: 'Emissions per capita (Kilograms of carbon dioxide)'
  Número de observações: 1722
  Média: 4.44
  Mediana: 2.50
  Desvio Padrão: 5.52
  Variância: 30.45
  Amplitude (Máx - Mín): 45.40


# No DataFrame emissões de carbono
## Coluna: emissão estimada de CO2
### Média: 
O quanto o país médio polui em 1 ano
### Mediana:
- Observar junto à média
    - Se ela for maior que a média, pode-se concluir que muitos países contribuem expressivamente para as emissões, consequentemente, deve-se haver países que contribuem pouquíssimo para ela
    - Se ela for menor que a média, pode-se concluir que poucos países contribuem expressivamente para as emissões, demonstrando altos focos poluentes
### Desvio padrão
- Mostra o quanto os países variam em média em suas estatísticas de poluição
### Variãncia
- A média de como os países se distanciam da média
    - Uma alta variância significa que ou os países poluem muito, ou poluem pouco
    - Uma baixa variância significa que a emissão de CO2 é bem distribuída no mundo
### Amplitude
- Nos mostra como os países estão distrbuídos em relação a como poluem
    - Uma amplitude menor que a média mostra que os países poluem na mesma proporção
    - Uma amplitude maior significa que poucos países poluem muito

## Coluna:Emissões per capita
### Média:
### Mediana
### Desvio padrão
### Variância
### Amplitude


In [10]:
def gerar_relatorio_gdp_estatisticas(filepath):
    try:
        df = pd.read_csv(filepath, encoding='utf-8')
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {filepath}")
        return
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return

    colunas_analise = ['Agricultura', 'Indústria', 'Serviços']

    # Verificar se as colunas existem
    for coluna in colunas_analise:
        if coluna not in df.columns:
            print(f"Erro: Coluna '{coluna}' não encontrada no CSV.")
            return

    print("--- Resultados Estatísticos ---")

    for coluna in colunas_analise:
        # Limpeza e conversão para numérico
        # Remove '%', substitui ',' por '.', depois converte para numérico
        df[coluna] = df[coluna].astype(str).str.rstrip('%').str.replace(',', '.', regex=False)
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')

        # Calcular estatísticas
        media = df[coluna].mean()
        mediana = df[coluna].median()
        desvio_padrao = df[coluna].std()
        variancia = df[coluna].var()
        
        if df[coluna].notna().any():
            amplitude = df[coluna].max() - df[coluna].min()
        else:
            amplitude = float('nan') 

        print(f"\nEstatísticas para a coluna: '{coluna}'")
        print(f"  Média: {media:.2f}" if pd.notna(media) else "  Média: N/A")
        print(f"  Mediana: {mediana:.2f}" if pd.notna(mediana) else "  Mediana: N/A")
        print(f"  Desvio Padrão: {desvio_padrao:.2f}" if pd.notna(desvio_padrao) else "  Desvio Padrão: N/A")
        print(f"  Variância: {variancia:.2f}" if pd.notna(variancia) else "  Variância: N/A")
        print(f"  Amplitude (Máx - Mín): {amplitude:.2f}" if pd.notna(amplitude) else "  Amplitude (Máx - Mín): N/A")

if __name__ == "__main__":
    arquivo_csv_gdp = "gdp_composicao_setores.csv"
    gerar_relatorio_gdp_estatisticas(arquivo_csv_gdp)



--- Resultados Estatísticos ---

Estatísticas para a coluna: 'Agricultura'
  Média: 10.99
  Mediana: 6.60
  Desvio Padrão: 12.08
  Variância: 145.86
  Amplitude (Máx - Mín): 60.70

Estatísticas para a coluna: 'Indústria'
  Média: 25.47
  Mediana: 24.30
  Desvio Padrão: 12.69
  Variância: 160.94
  Amplitude (Máx - Mín): 61.40

Estatísticas para a coluna: 'Serviços'
  Média: 61.73
  Mediana: 61.10
  Desvio Padrão: 16.40
  Variância: 269.03
  Amplitude (Máx - Mín): 100.00


## No dataframe de composição do PIB dos países
### A média das colunas 
- representa o quanto, no mundo, esses setores contribuem para o PIB
### A mediana das colunas 
 - nos mostra como estão distribuídos esses focos de setores:
    - Uma mediana menor que a média significa que poucos países detém muita força em certos setores
    - Uma mediana maior que a média significa que muitos países investem nesse setor, como podemos ver nos serviços
### O desvio padrão das colunas
 - Nos mostra o quanto os países variam no investimento em tal setor
    - Um desvio padrão alto significa que há mais países com contribuições desiguais
    - Um desvio padrão baixo significa que os dados são mais homogêmeos
### A variância das colunas
 - Nos mostra como, em média, os países estão próximos à média
    - Um valor mais alto na variância significa que os países tendem mais a possuírem investimentos fora da média
    - Um valor mais baixo significa que o setor possui, em média, presença sustentável nos países
        - Se espera que setores mais estáveis, como a saúde, tenham variâncias menores
### A amplitude das colunas
- Nos mostra como o país que mais investe num setor se compara ao que menos investe
    - Uma amplitude alta significa que há países que há disparidade entre investimentos no setor entre países
    - Uma amplitude baixa significa que o setor é mais comum de possuir investimentos, espera-se amplitudes baixas em setores como transporte público, saneamento básico, fornecimento de energia, etc
